In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from getpass import getpass
import os
import pandas as pd
from pathlib import Path
import quandl
from sklearn.decomposition import PCA

In [6]:
quandl_api_key = getpass('Quandl API Key')

Quandl API Key ····················


In [7]:
def get_rates():
    cache_file = Path("rates.csv")
    if cache_file.exists():
        return pd.read_csv(cache_file).set_index(["Date"])

    rates = quandl.get(
        [f"FRED/DGS{tenor}" for tenor in [2, 5, 7, 10, 30]],
        start_date="2000-01-01",
        collapse="monthly",
        transform="diff",
        authtoken=quandl_api_key
    ).dropna()
    rates.to_csv(cache_file)
    return rates

In [8]:
rates = get_rates()

In [9]:
pca = PCA()
pca.fit(rates)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [10]:
print(pca.components_)
print(pca.explained_variance_ratio_)

[[-0.35058342 -0.47315189 -0.49481541 -0.48693251 -0.41385141]
 [ 0.64220107  0.29231193  0.05280315 -0.23310326 -0.66708786]
 [-0.63465824  0.2815649   0.37866284  0.26671034 -0.55082681]
 [ 0.20940748 -0.40728039 -0.30362855  0.78846213 -0.27642063]
 [ 0.1342627  -0.66747653  0.71888364 -0.12550961 -0.06246805]]
[0.86503327 0.10495049 0.02417499 0.00408606 0.00175519]
